In [8]:
import pandas as pd
df = pd.read_csv(r'F:\Yatharth Joshi\ML FI TASK 1\Sample - Superstore.csv',encoding='cp1252')
df['Order Date'] = pd.to_datetime(df['Order Date'])
print("Data loaded successfully!")
print(df.head())

Data loaded successfully!
   Row ID        Order ID Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156 2016-11-08  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156 2016-11-08  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688 2016-06-12   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966 2015-10-11  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966 2015-10-11  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \


In [15]:
daily_sales = df.groupby('Order Date')['Sales'].sum().reset_index()
print(daily_sales.head())

  Order Date     Sales
0 2014-01-03    16.448
1 2014-01-04   288.060
2 2014-01-05    19.536
3 2014-01-06  4407.100
4 2014-01-07    87.158


Columns after renaming: Index(['ds', 'y'], dtype='object')
Data head after renaming:
          ds         y
0 2014-01-03    16.448
1 2014-01-04   288.060
2 2014-01-05    19.536
3 2014-01-06  4407.100
4 2014-01-07    87.158


In [18]:
import pandas as pd
# Assuming 'df' is your loaded and cleaned DataFrame

# 4. Categorical Data Aggregation (For Power BI Filters)
# Action: Create a separate, aggregated table that keeps categorical dimensions 
# (like Region, Category) along with the monthly sales.

# Replace 'M' with 'ME' to fix the FutureWarning
monthly_region_sales = df.groupby(
    [pd.Grouper(key='Order Date', freq='ME'), 'Region'] # <- Change 'M' to 'ME'
)['Sales'].sum().reset_index()

monthly_region_sales.rename(columns={'Order Date': 'Date', 'Sales': 'Actual Sales'}, inplace=True)
monthly_region_sales.to_csv('monthly_region_sales.csv', index=False)

print("Phase 1, Step 4 completed without warnings. File 'monthly_region_sales.csv' exported.")
print(monthly_region_sales.head())

Phase 1, Step 4 completed without warnings. File 'monthly_region_sales.csv' exported.
        Date   Region  Actual Sales
0 2014-01-31  Central      1539.906
1 2014-01-31     East       436.174
2 2014-01-31    South      9322.092
3 2014-01-31     West      2938.723
4 2014-02-28  Central      1233.174


In [20]:

from prophet import Prophet

# Initialize Prophet
m = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    seasonality_mode='multiplicative'
)

# Fit the model (This will now work!)
m.fit(daily_sales)

print("Prophet model successfully fit to data.")

16:24:46 - cmdstanpy - INFO - Chain [1] start processing
16:24:47 - cmdstanpy - INFO - Chain [1] done processing


Prophet model successfully fit to data.


In [22]:
# Create 180 future periods (days)
future = m.make_future_dataframe(periods=180)
# Generate the forecast (yhat is the prediction)
forecast = m.predict(future)

# To specifically see the new 180 future predictions:
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10))

             ds         yhat   yhat_lower   yhat_upper
1407 2018-06-19  1466.584709 -1227.438661  4271.743327
1408 2018-06-20   363.554653 -2435.275641  2876.468410
1409 2018-06-21  2091.999399  -611.455780  4790.201886
1410 2018-06-22  2474.179018  -187.403904  5380.596448
1411 2018-06-23  1894.296058  -799.873540  4561.913878
1412 2018-06-24  2151.785777  -622.657431  4980.873205
1413 2018-06-25  2393.299019  -464.300789  5040.741735
1414 2018-06-26  1430.374990 -1383.988544  4163.410084
1415 2018-06-27   336.797810 -2263.235441  3209.779814
1416 2018-06-28  2080.236751  -577.679220  4572.873720


In [23]:
# Select key columns from forecast
final_forecast_data = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Merge with actual historical data
combined_df = final_forecast_data.merge(daily_sales, on='ds', how='left')

# Create the 'Data Type' flag (Actual or Forecast)
combined_df['Data Type'] = combined_df['y'].apply(lambda x: 'Actual' if pd.notna(x) else 'Forecast')

# Fill NaN values in 'y' with the forecast for a seamless line
combined_df['Sales Value'] = combined_df['y'].fillna(combined_df['yhat'])

# Export the final file
combined_df.to_csv('final_sales_forecast.csv', index=False)